In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from source.TrainDataGeneration import append_ORD_TM,append_ORD_VOL,append_TM_GP,GetGroupDataFrame,append_STEP5
from DataInfo import ISU_list_MBRN_sum_top,MBRN_sum_top,header_df,env_columns



ISUlist_init=['KR7071930002',
 'KR7040670002',
 'KR7044180008',
 'KR7052420007',
 'KR7043100007',
 'KR7024810004',
 'KR7036540003',
 'KR7058820002',
 'KR7018700005',
 'KR7043200005']
ISUlist_init=ISU_list_MBRN_sum_top
MBR, BRN =42,1
groupmin=10
datasubfix="1111Train_08"
PathDir = '/Data/ksqord/'
EnvPathDir=PathDir+'EnvData/'
TrainPathDir='/Data/LOBData/TrainData/'

In [2]:
columns = header_df
env_columns = env_columns
ord_columns = ['ISU_CD', 'ORD_DD', 'ORD_ACPT_NO', 'BLKTRD_TP_CD', '호가장처리가격', 'ASKBID_TP_CD', 'MODCANCL_TP_CD',
 'ORD_TP_CD', 'ORD_COND_CD', 'INVST_TP_CD', 'ORD_ACPT_TM', 'ORD_QTY', 'ORD_PRC', '호가우선순위번호', 'MBR_NO', 'BRN_NO',
 'CNTR_CD', 'TRST_PRINC_TP_CD', 'FORNINVST_TP_CD', 'ORD_MEDIA_TP_CD', '예상체결가격', '예상체결수량', '호가체결접수순서번호',
 '누적체결수량', '누적거래대금', 'AGG_TM', 'ORGN_ORD_ACPT_NO', '자동취소처리구분코드', 'PT_TP_CD']
CancelCount=True
feat_cols=['매도5단계호가합계잔량', '매수5단계호가합계잔량', '매도10단계호가합계잔량',
       '매수10단계호가합계잔량', '매도총호가잔량', '매수총호가잔량', '고가', '저가',
           '시가', '직전체결가격','NET_ORD_QTY2']
# feat_cols=['매도총호가잔량', '매수총호가잔량', '고가', '저가', 
#            '직전체결가격','NET_ORD_QTY2']


TotDF=pd.DataFrame()

FileList=os.listdir(PathDir)
FileList.sort()
FileList= [ file for file in FileList if ".csv" in file ]

for filename in FileList[:10]:
    envfilename = filename[:-4]+"_10min_last.csv"
    print(filename)
    Data=pd.read_csv(os.path.join(PathDir,filename),names=header_df)
    Data=append_ORD_TM(Data)
    Data=append_ORD_VOL(Data)
    Data=append_TM_GP(Data,groupmin=groupmin)
    Data['NET_ORD_QTY']=Data['ORD_QTY']*(2*Data['ASKBID_TP_CD']-3)*(2-Data['MODCANCL_TP_CD'])
    Data['NET_ORD_VOL']=Data['NET_ORD_QTY']*(Data['ORD_PRC'] + Data['직전체결가격']*(Data['ORD_PRC'] == 0))

    EnvData=pd.read_csv(EnvPathDir+envfilename)
    ISU_list= [a for a in ISUlist_init  if a in EnvData.ISU_CD.unique() and a in Data.ISU_CD.unique() ]
    print(ISU_list)
    print(len(ISU_list))
    if BRN is not None:
        Data_MBRN=Data[(Data['MBR_NO']==MBR) & (Data['BRN_NO']==BRN)]
    else:
        Data_MBRN=Data[(Data['MBR_NO']==MBR)]
    Data_MBRN=Data_MBRN[Data_MBRN['ISU_CD'].isin(ISU_list)]

    groupcolumns=['ORD_DD','ISU_CD','TM_GP']
    sumcolumns=['NET_ORD_QTY','NET_ORD_VOL']

    meancolumns=['ORD_PRC']
    lastcolumns=[]

    GDF=GetGroupDataFrame(Data_MBRN,groupcolumns,sumcolumns,meancolumns,lastcolumns)
    GDF.set_index(['ORD_DD','ISU_CD', 'TM_GP'], inplace=True)
    GDF = GDF.reindex(pd.MultiIndex.from_product([GDF.index.levels[0],GDF.index.levels[1],list(range(-9,55))]))
    GDF = GDF.fillna(0)
    GDF = GDF.reset_index()
    GDF=GDF.rename(columns={'level_2': 'TM_GP'})


    ISU_list= [a for a in GDF.ISU_CD.unique() if a in EnvData.ISU_CD.unique() ]
    EnvData_MBRN=EnvData[EnvData['ISU_CD'].isin(ISU_list)]

    TrainData = pd.concat([GDF.set_index(['ORD_DD','ISU_CD', 'TM_GP']),EnvData_MBRN.set_index(['ORD_DD','ISU_CD', 'TM_GP'])],axis=1)
    TrainData=TrainData.reset_index()
    TrainData=append_STEP5(TrainData)
    TrainData['10단계호가합계잔량']=TrainData['매수10단계호가합계잔량']+TrainData['매도10단계호가합계잔량']
    TrainData["NET_ORD_QTY2"]=(TrainData["NET_ORD_QTY"]>0).replace({True: 1, False: 0})-(TrainData["NET_ORD_QTY"]<0).replace({True: 1, False: 0})

    Train_df=TrainData[(TrainData['TM_GP']>=0) & (TrainData['TM_GP']<39)]



    divdict={}
    loglist=[]
    loglist+=['고가','저가','직전체결가격']
    minmaxnormlist=[]
    minmaxnormlist+=['매도총호가잔량', '매수총호가잔량', '고가', '저가', 
               '직전체결가격']

    for divcol in divdict:
        Train_df[divdict[divcol]]=Train_df[divdict[divcol]].div(Train_df[divcol],axis=0).values
    Train_df[loglist]=np.log(Train_df[loglist])
    Train_df[minmaxnormlist]=(Train_df[minmaxnormlist]-Train_df[minmaxnormlist].min())/(Train_df[minmaxnormlist].max()\
                                                                                        -Train_df[minmaxnormlist].min())
    TotDF=TotDF.append(Train_df[feat_cols])

ksqord_20170601.csv


/root/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


['KR7223310004', 'KR7161580006', 'KR7060260007', 'KR7024840001', 'KR7035720002', 'KR7049180003', 'KR7217270008', 'KR7145020004', 'KR7083660001', 'KR7031860000', 'KR7030270003', 'KR7192080000', 'KR7197210008', 'KR7068270008', 'KR7014940001', 'KR7053030003', 'KR7201490000', 'KR7086900008', 'KR7086390002', 'KR7025440009', 'KR7064260003', 'KR7036930006', 'KR7067170001', 'KR7033830001', 'KR7068790005', 'KR7065060006', 'KR7102940004', 'KR7091590000', 'KR7035480003', 'KR7008800005', 'KR7050110006', 'KR7114570005', 'KR7036690006', 'KR7240810002', 'KR7178920005', 'KR7130960008', 'KR7034230003', 'KR7036830008', 'KR7051370005', 'KR7043580000', 'KR7090360009', 'KR7046890000', 'KR7204840003', 'KR7131370009', 'KR7056190002', 'KR7001840008', 'KR7224060004', 'KR7033500000', 'KR7078150000', 'KR7090710005', 'KR7067390005', 'KR7064240005', 'KR7108790007', 'KR7212560007', 'KR7006920003', 'KR7049120009', 'KR7022100002', 'KR7109740001', 'KR7100130004', 'KR7215600008', 'KR7013810007', 'KR7200230001', 'KR7048

/root/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


ksqord_20170602.csv
['KR7223310004', 'KR7161580006', 'KR7060260007', 'KR7024840001', 'KR7035720002', 'KR7049180003', 'KR7217270008', 'KR7145020004', 'KR7083660001', 'KR7031860000', 'KR7030270003', 'KR7192080000', 'KR7197210008', 'KR7068270008', 'KR7014940001', 'KR7053030003', 'KR7201490000', 'KR7086900008', 'KR7086390002', 'KR7025440009', 'KR7064260003', 'KR7036930006', 'KR7067170001', 'KR7033830001', 'KR7068790005', 'KR7065060006', 'KR7102940004', 'KR7091590000', 'KR7035480003', 'KR7008800005', 'KR7050110006', 'KR7114570005', 'KR7036690006', 'KR7240810002', 'KR7178920005', 'KR7130960008', 'KR7034230003', 'KR7036830008', 'KR7051370005', 'KR7043580000', 'KR7090360009', 'KR7046890000', 'KR7204840003', 'KR7131370009', 'KR7056190002', 'KR7001840008', 'KR7224060004', 'KR7033500000', 'KR7078150000', 'KR7090710005', 'KR7067390005', 'KR7064240005', 'KR7108790007', 'KR7212560007', 'KR7006920003', 'KR7049120009', 'KR7022100002', 'KR7109740001', 'KR7100130004', 'KR7215600008', 'KR7013810007', 'KR

ksqord_20170613.csv
['KR7223310004', 'KR7161580006', 'KR7060260007', 'KR7024840001', 'KR7035720002', 'KR7049180003', 'KR7217270008', 'KR7145020004', 'KR7083660001', 'KR7031860000', 'KR7030270003', 'KR7192080000', 'KR7197210008', 'KR7068270008', 'KR7014940001', 'KR7053030003', 'KR7201490000', 'KR7086900008', 'KR7086390002', 'KR7025440009', 'KR7064260003', 'KR7036930006', 'KR7067170001', 'KR7033830001', 'KR7068790005', 'KR7065060006', 'KR7102940004', 'KR7091590000', 'KR7035480003', 'KR7008800005', 'KR7050110006', 'KR7114570005', 'KR7036690006', 'KR7240810002', 'KR7178920005', 'KR7130960008', 'KR7034230003', 'KR7036830008', 'KR7051370005', 'KR7043580000', 'KR7090360009', 'KR7046890000', 'KR7204840003', 'KR7131370009', 'KR7056190002', 'KR7001840008', 'KR7224060004', 'KR7033500000', 'KR7078150000', 'KR7090710005', 'KR7067390005', 'KR7064240005', 'KR7108790007', 'KR7212560007', 'KR7006920003', 'KR7049120009', 'KR7022100002', 'KR7109740001', 'KR7100130004', 'KR7215600008', 'KR7013810007', 'KR

In [3]:
DataSubfix = str(MBR) + '_' + str(BRN) + datasubfix
XDataname = 'Train_ORD' + '_' + DataSubfix + '.npy'
YDataname = 'Train_ORD_Label_' + '_' + DataSubfix + '.npy'

In [4]:
TrainPathDir+XDataname

'/Data/LOBData/TrainData/Train_ORD_42_11111Train_08.npy'

In [5]:
Train_df[feat_cols].values.dtype
Train_df[feat_cols].values.dtype

dtype('float64')

In [6]:
Train_df[feat_cols[-1]].values.dtype

dtype('int64')

In [12]:
print(TotDF[feat_cols[:-1]].shape)
print(TotDF[feat_cols[-1]].shape)

(36465, 10)
(36465,)


In [14]:
TotDF[feat_cols[-1]].values

array([-1, -1, -1, ...,  1,  1, -1])

In [13]:
np.save(TrainPathDir+XDataname,TotDF[feat_cols[:]].values.astype('float64'))
np.save(TrainPathDir+YDataname,TotDF[feat_cols[-1]].values)
print("path:",TrainPathDir+XDataname)
print("path:",TrainPathDir+YDataname)
    

path: /Data/LOBData/TrainData/Train_ORD_42_11111Train_08.npy
path: /Data/LOBData/TrainData/Train_ORD_Label__42_11111Train_08.npy
